# Search Engine Based on Counting In-Links

This search engine algorithm is based on counting in-links and recovering all sink node and sink region by adding a linkage towards other parts of the internet. Then, we define random walk to be the process where we randomly enter through a page on the internet and visit the next page that is connected to the entrance randomly.

In [26]:
import numpy as np

In [27]:
# Build an adjacency matrix based on whether the two points are connected

A = np.array([[0,1,0,0,1,0],
     [1,0,1,0,1,0],
     [0,0,0,0,1,0],
     [0,0,1,0,0,1],
     [0,1,0,1,0,0],
     [0,0,0,0,0,0]])
A

array([[0, 1, 0, 0, 1, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [28]:
# count-in links
def count_out_links(A):
    c = A.sum(axis=0)
    return c
count_out_links(A)

array([1, 2, 2, 1, 3, 1])

There are potentials that people would use the connections to create a sink node or sink region such that make our algorithm not working. Therefore, we need to use an algorithm to handle that situation.

In [29]:
# Find all sink nodes in the adjacency matrix
def revise_sink_nodes(A):
    temp = count_out_links(A)
    A[:,np.where(temp==0)] += 1
    return A

In [30]:
# Compute probability matrix from adjacency matrix
def compute_P(A):
    temp = count_out_links(A)
    P = (A@np.diag(1/temp))
    return P


In [31]:
# Compute the proper probability matrix after we make the sink node not sink anymore
def compute_S(A):
    A = revise_sink_nodes(A)
    S = compute_P(A)
    return S

In [32]:
# Using the formula to generate the final probability matrix
def compute_G(S, alpha=0.95):
    G = alpha*S+((1-alpha)/S.shape[0])*np.ones_like(S)
    return G

In [33]:
# Taking random walk for one step
def random_walk_one_step(P, x):
    y = P@x
    return y

In [34]:
# Determine whether two np array equal to each other
def all_close(x, y, tol=0.01):
    c = np.allclose(x,y,atol=tol)
    return c


In [35]:
# Final random walk with default maximum as 100 steps
def random_walk(P, x, tol=0.01, max_steps=100):
    for i in range(max_steps):
        if all_close(random_walk_one_step(P, x), x, tol):
            break
        x = random_walk_one_step(P, x)
    return x

In [36]:
# Assemble the search engine
def search_engine(A):
    n = A.shape[0]  
    x = np.ones(n)/n
    G = compute_G(A)
    s = random_walk(G,x)
    s = np.argsort(s)[::-1]
    return s
search_engine_v1(A)

array([1, 0, 4, 2, 3, 5], dtype=int64)